In [ ]:
import os
import time
import re
import string
from string import punctuation
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

def remove_ads_str(text):
  ads_list = ['\nĐể đặt lịch', '\nTại BVĐK Tâm Anh', '\nTrung tâm', '\nChuyên khoa', '\nNếu cần đặt lịch']
  for ad in ads_list:
    if ad.lower() in text.lower():
      process_paragraph = re.sub(ad.lower() + '.*', '', text.lower())
      text = text[ : len(process_paragraph)].strip()
    else:
      text = text.strip()
  return text

def preprocessing_text(text):
  text = remove_ads_str(text).strip()
  text = re.sub('\(\d+\)', ' ', text)
  text = text.replace('\n', ' ')
  text = text.replace('\xa0', ' ')
  text = text.replace('\u200b', ' ')
  text = re.sub(' +', ' ', text).strip()
  return text

def get_content(url):
  html_text = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(html_text.text, "html.parser")
  div = soup.find("div", {"id": "ftwp-postcontent"})
  ad = div.find("div", {"class": "alert alert-info"})
  new_soup = BeautifulSoup(str(div).split(str(ad))[0], "html.parser")
  title = soup.find('h1').text.split(':')[0]
  new_str = ''
  for i in re.findall(r'<\w.*?> +?', str(new_soup).replace('<strong>','').replace('</strong>',',')):
    tag_name = str(i).split()[0]
    for j in  ['<p', '<h2', '<ul', '<li', '<h3', '<h4']:
      if j in str(i).split()[0]:
        if BeautifulSoup(i, "html.parser").text != ' ' and tag_name == '<h2':
          new_str += tag_name + '>' + BeautifulSoup(i, "html.parser").text + ' <key> ' + '\n'
        else:
          new_str +=  BeautifulSoup(i, "html.parser").text +'\n'
  str_list = [title + '. ' + preprocessing_text(text) for text in new_str.split('<h2>')[1:]]
  first_str = ['Định nghĩa ' + title + ' <key> ' + preprocessing_text(new_str.split('<h2>')[0])]
  return first_str + str_list

def find_h3_text(url):
  html_text = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(html_text.text, "html.parser")
  div = soup.find("div", {"id": "ftwp-postcontent"})
  ad = div.find("div", {"class": "alert alert-info"})
  new_soup = BeautifulSoup(str(div).split(str(ad))[0], "html.parser")
  title = soup.find('h1').text.split(':')[0]
  new_str = ''
  for i in re.findall(r'<\w.*?> +?', str(new_soup).replace('<strong>','').replace('</strong>',',')):
    tag_name = str(i).split()[0]
    for j in  ['<p','<ul', '<li', '<h3', '<h4']:
      if j in str(i).split()[0]:
        if BeautifulSoup(i, "html.parser").text != ' ' and tag_name == '<h3':
          new_str += tag_name + '>' + BeautifulSoup(i, "html.parser").text + ' <key> ' + '\n'
        else:
          new_str +=  BeautifulSoup(i, "html.parser").text +'\n'
  str_list = [title + '. ' + preprocessing_text(text) for text in new_str.split('<h3>')[1:]]
  first_str = [title + ' <key> ' + preprocessing_text(new_str.split('<h3>')[0])]
  return first_str + str_list

def find_h4_text(url):
  html_text = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(html_text.text, "html.parser")
  div = soup.find("div", {"id": "ftwp-postcontent"})
  ad = div.find("div", {"class": "alert alert-info"})
  new_soup = BeautifulSoup(str(div).split(str(ad))[0], "html.parser")
  title = soup.find('h1').text.split(':')[0]
  new_str = ''
  for i in re.findall(r'<\w.*?> +?', str(new_soup).replace('<strong>','').replace('</strong>',',')):
    tag_name = str(i).split()[0]
    for j in  ['<p', '<ul', '<li', '<h4']:
      if j in str(i).split()[0]:
        if BeautifulSoup(i, "html.parser").text != ' ' and tag_name == '<h4':
          new_str = tag_name + '>' + BeautifulSoup(i, "html.parser").text + ' <key> ' + '\n'
          sub_graph = str(new_soup).split(i)[1]
          new_str += BeautifulSoup(sub_graph, "html.parser").text
  if len(new_str.split('<h4>')) > 1 and new_str.split('<h4>')[0] != '':
    str_list = [title + '. ' + preprocessing_text(text) for text in new_str.split('<h4>')[1:]]
    first_str = [title + '. ' + preprocessing_text(new_str.split('<h4>')[0])]
    return first_str + str_list
  else:
    return title + '. ' + preprocessing_text(new_str).replace('<h4>', '')

def get_info(path):
  import re
  info_dict = {}
  with open(path,'r', encoding="utf-8") as f:
    content = f.read()
  info = re.split(r'<h1>',content)[0].split('\n')
  info_dict['url'] = info[0]
  return info_dict

def get_url(corpus_path):
  url_dict = {}
  for article in os.listdir(corpus_path):
    new_dir = os.path.join(corpus_path, article)
    try:
      url_dict[article] = get_info(new_dir)
    except:
      print(f'error: {article}')
      url_dict[article] = 'null'
  return url_dict

def get_corpus_dict(url_dict):
  h2_corpus_dict = {}
  h3_corpus_dict = {}
  h4_corpus_dict = {}
  h2_error_list = []
  h3_error_list = []
  h4_error_list = []
  for key in url_dict.keys():
    time.sleep(1)
    url = url_dict[key]['url']
    try:
      h2_corpus_dict[key] = get_content(url)
      print('h2_finished: ', key)
    except:
      h2_error_list.append(key)
      print('error: ', key)
    try:
      h3_corpus_dict[key] = find_h3_text(url)
      print('h3_finished: ', key)
    except:
      h3_error_list.append(key)
      print('error: ', key)
    try:
      h4_corpus_dict[key] = find_h4_text(url)
      print('h4_finished: ', key)
    except:
      h4_error_list.append(key)
      print('error: ', key)
  return h2_corpus_dict, h3_corpus_dict, h4_corpus_dict, h2_error_list, h3_error_list, h4_error_list

def get_h2_corpus_dict(h2_dict):
  new_answer_dict = {}
  for key in h2_dict.keys():
    content = h2_dict[key]
    for ct in content:
      spt = ct.split('<key>')
      quest = spt[0].strip()
      if ' là gì? nguyên nhân, triệu chứng và cách điều trị' in quest.lower():
        quest = ''.join(quest.lower().split(' là gì? nguyên nhân, triệu chứng và cách điều trị'))
      quest = re.sub(' +', ' ', quest)
      context = ct.replace(' <key> ', ' ').strip()
      context = re.sub(r'\[\d+\]', ' ', context)
      context = re.sub(r"[0-9]\.", ' ', context)
      context = context.replace('\u200b', ' ')
      context = context.replace('\xa0', ' ')
      context = re.sub(' +', ' ', context)
      context = context.replace(' là gì? nguyên nhân, triệu chứng và cách điều trị', ' ')
      if len(spt) > 1:
        new_answer_dict[quest] = context.strip()
  return new_answer_dict

def get_h3_corpus_dict(h3_dict):
  final_h3_corpus_dict = {}
  for key in h3_dict.keys():
    content = h3_dict[key]
    for ct in content:
      spt = ct.split('<key>')
      if len(spt) > 1 and spt[1] != '':
        quest = spt[0].strip()
        if ' là gì? nguyên nhân, triệu chứng và cách điều trị' in quest.lower():
          quest = ''.join(quest.lower().split(' là gì? nguyên nhân, triệu chứng và cách điều trị'))
        quest = re.sub(' +', ' ', quest)
        context = spt[1].replace(' <key> ', ' ').strip()
        context = re.sub(r'\[\d+\]', ' ', context)
        context = re.sub(r"[0-9]\.", ' ', context)
        context = context.replace('\u200b', ' ')
        context = context.replace('\xa0', ' ')
        context = re.sub(' +', ' ', context)
        context = context.replace(' là gì? nguyên nhân, triệu chứng và cách điều trị', ' ')
        final_h3_corpus_dict[quest] = context.strip()
  return final_h3_corpus_dict

def get_h4_corpus_dict(h4_dict):
  final_h4_corpus_dict = {}
  for key, value in h4_dict.items():
    if '<key>' in value:
      spt = value.split('<key>')
      if len(spt) > 1 and spt[1] != '':
        quest = spt[0].strip()
        if ' là gì? nguyên nhân, triệu chứng và cách điều trị' in quest.lower():
          quest = ''.join(quest.lower().split(' là gì? nguyên nhân, triệu chứng và cách điều trị'))
        quest = re.sub(' +', ' ', quest)
        context = value.replace(' <key> ', ' ').strip()
        context = re.sub(r'\[\d+\]', ' ', context)
        context = re.sub(r"[0-9]\.", ' ', context)
        context = context.replace('\u200b', ' ')
        context = context.replace('\xa0', ' ')
        context = re.sub(' +', ' ', context)
        context = context.replace(' là gì? nguyên nhân, triệu chứng và cách điều trị', ' ')
        final_h4_corpus_dict[quest] = context.strip()
  return final_h4_corpus_dict

def write_csv(corpus_dict, dict_name):
  with open(dict_name+'.csv', mode='w', encoding='utf-8') as csv_file:
    fieldnames = ['topic', 'context']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for key in corpus_dict.keys():
      writer.writerow({'topic': key, 'context': corpus_dict[key]})

In [ ]:
#Download corpus
!gdown -q 1mbfJWmB_UPfKd1k4X9v4-vHV1skhD66R

In [ ]:
#Unzip
!unzip -q '/content/corpus.zip' -d '/content/'

In [ ]:
corpus_path = '/content/corpus'
url_dict = get_url(corpus_path)
h2_corpus_dict, h3_corpus_dict, h4_corpus_dict, h2_error_list, h3_error_list, h4_error_list = get_corpus_dict(url_dict)

In [ ]:
h2_dct = get_h2_corpus_dict(h2_corpus_dict)
write_csv(h2_dct, dict_name='corpus_dict_h2')
h3_dct = get_h3_corpus_dict(h3_corpus_dict)
write_csv(h3_dct, dict_name='corpus_dict_h3')
h4_dct = get_h4_corpus_dict(h4_corpus_dict)
write_csv(h4_dct, dict_name='corpus_dict_h4')

In [ ]:
h2_df = pd.read_csv('corpus_dict_h2.csv')
h3_df = pd.read_csv('corpus_dict_h3.csv')
h4_df = pd.read_csv('corpus_dict_h4.csv')
data = pd.concat([h2_df, h3_df, h4_df], ignore_index=True, axis=0)
data.to_csv('ViMed_corpus_dict.csv', index=False)

In [ ]:
len(data)

10251

In [ ]:
pd.read_csv('ViMed_corpus_dict.csv')

,topic,context
0,Định nghĩa Mắt viễn thị,"Định nghĩa Mắt viễn thị Viễn thị, (Hyperopia) ..."
1,Mắt viễn thị. Viễn thị ảnh hưởng đến tầm nhìn ...,Mắt viễn thị. Viễn thị ảnh hưởng đến tầm nhìn ...
2,Mắt viễn thị. Nguyên nhân viễn thị ở mắt và yế...,Mắt viễn thị. Nguyên nhân viễn thị ở mắt và yế...
3,Mắt viễn thị. Dấu hiệu viễn thị ở mắt,Mắt viễn thị. Dấu hiệu viễn thị ở mắt Người bệ...
4,Mắt viễn thị. Viễn thị có chữa được không?,Mắt viễn thị. Viễn thị có chữa được không? Viễ...
...,...,...
10246,Tắc ống dẫn tinh. 5. Tắc ống phóng tinh,Tắc ống dẫn tinh. Tắc ống phóng tinh Điều trị ...
10247,"U tuyến giáp là gì? Dấu hiệu, nguyên nhân, biế...","U tuyến giáp là gì? Dấu hiệu, nguyên nhân, biế..."
10248,Lupus ban đỏ dạng đĩa. Sinh thiết da,Lupus ban đỏ dạng đĩa. Sinh thiết da Sinh thiế...
10249,Tinh hoàn lạc chỗ. 2.2 Hướng điều trị:,Tinh hoàn lạc chỗ. 2 Hướng điều trị: Khi tinh ...
